Title: Coffee quality prediction

Name: Shafaat Siddhi <br>
ID: 19301027 <br>
Course: CSE 427 <br>
Sec: 1

In [ ]:
# coffee quality prediction
# import data
# Visualize data
# select test and train data
# Modeling using linear reg, logistics reg, decision tree, random forest

Importing Libraries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import accuracy_score

Importing Data:

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/427 final/merged_data_cleaned.csv')

df.head(3)

,Unnamed: 0,Species,Owner,Country.of.Origin,Farm.Name,Lot.Number,Mill,ICO.Number,Company,Altitude,...,Color,Category.Two.Defects,Expiration,Certification.Body,Certification.Address,Certification.Contact,unit_of_measurement,altitude_low_meters,altitude_high_meters,altitude_mean_meters
0,0,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,0,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
1,1,Arabica,metad plc,Ethiopia,metad plc,NaN,metad plc,2014/2015,metad agricultural developmet plc,1950-2200,...,Green,1,"April 3rd, 2016",METAD Agricultural Development plc,309fcf77415a3661ae83e027f7e5f05dad786e44,19fef5a731de2db57d16da10287413f5f99bc2dd,m,1950.0,2200.0,2075.0
2,2,Arabica,grounds for health admin,Guatemala,"san marcos barrancas ""san cristobal cuch",NaN,NaN,NaN,NaN,1600 - 1800 m,...,NaN,0,"May 31st, 2011",Specialty Coffee Association,36d0d00a3724338ba7937c52a378d085f2172daa,0878a7d4b9d35ddbf0fe2ce69a2062cceb45a660,m,1600.0,1800.0,1700.0


Cleaning and electing columns:

In [ ]:

# selecting needed columns
columns = ['Total.Cup.Points', 'Species','Country.of.Origin','Processing.Method', 'Aroma', 'Aftertaste','Acidity','Body', 'Balance', 'Uniformity', 'Moisture', 'altitude_mean_meters']

# changing data frame to selected columns
df = df[columns]
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1339 entries, 0 to 1338
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Total.Cup.Points      1339 non-null   float64
 1   Species               1339 non-null   object 
 2   Country.of.Origin     1338 non-null   object 
 3   Processing.Method     1169 non-null   object 
 4   Aroma                 1339 non-null   float64
 5   Aftertaste            1339 non-null   float64
 6   Acidity               1339 non-null   float64
 7   Body                  1339 non-null   float64
 8   Balance               1339 non-null   float64
 9   Uniformity            1339 non-null   float64
 10  Moisture              1339 non-null   float64
 11  altitude_mean_meters  1109 non-null   float64
dtypes: float64(9), object(3)
memory usage: 125.7+ KB


Handling Missing values:

In [ ]:
# checking missing values
df.isna().sum()

Total.Cup.Points          0
Species                   0
Country.of.Origin         1
Processing.Method       170
Aroma                     0
Aftertaste                0
Acidity                   0
Body                      0
Balance                   0
Uniformity                0
Moisture                  0
altitude_mean_meters    230
dtype: int64

In [ ]:
# dropping missing values
df = df.dropna()
df.isna().sum()

Total.Cup.Points        0
Species                 0
Country.of.Origin       0
Processing.Method       0
Aroma                   0
Aftertaste              0
Acidity                 0
Body                    0
Balance                 0
Uniformity              0
Moisture                0
altitude_mean_meters    0
dtype: int64

X and Y:

In [ ]:

Y = df['Total.Cup.Points']
Y

0       90.58
1       89.92
3       89.00
4       88.83
9       88.25
        ...  
1322    81.58
1327    81.25
1332    80.17
1333    79.33
1336    77.17
Name: Total.Cup.Points, Length: 1013, dtype: float64

In [ ]:
df_X = df.drop(columns = 'Total.Cup.Points')
df_X

,Species,Country.of.Origin,Processing.Method,Aroma,Aftertaste,Acidity,Body,Balance,Uniformity,Moisture,altitude_mean_meters
0,Arabica,Ethiopia,Washed / Wet,8.67,8.67,8.75,8.50,8.42,10.00,0.12,2075.0
1,Arabica,Ethiopia,Washed / Wet,8.75,8.50,8.58,8.42,8.42,10.00,0.12,2075.0
3,Arabica,Ethiopia,Natural / Dry,8.17,8.42,8.42,8.50,8.25,10.00,0.11,2000.0
4,Arabica,Ethiopia,Washed / Wet,8.25,8.25,8.50,8.42,8.33,10.00,0.12,2075.0
9,Arabica,Ethiopia,Natural / Dry,8.08,8.50,8.50,7.67,8.42,10.00,0.10,1822.5
...,...,...,...,...,...,...,...,...,...,...,...
1322,Robusta,India,Natural / Dry,7.67,7.75,7.50,7.75,7.58,10.00,0.11,1000.0
1327,Robusta,India,Natural / Dry,7.50,7.25,7.83,7.67,7.83,10.00,0.13,750.0
1332,Robusta,India,Natural / Dry,7.58,7.42,7.83,7.42,7.50,10.00,0.00,750.0
1333,Robusta,United States,Natural / Dry,7.92,7.42,7.42,7.42,7.42,9.33,0.00,3000.0


Non numeric features to numeric:

In [ ]:
# numeric column
numberic_col = ['Aroma','Aftertaste' ,'Acidity' ,'Balance' ,'Uniformity' ,'Moisture' ,'altitude_mean_meters']
std = StandardScaler()
x_std= std.fit_transform(df_X[numberic_col])
X_n = pd.DataFrame(x_std,columns=numberic_col)
X_n.head()


,Aroma,Aftertaste,Acidity,Balance,Uniformity,Moisture,altitude_mean_meters
0,3.666864,3.864168,3.981118,2.701608,0.293848,0.579691,0.028801
1,3.933036,3.352249,3.427644,2.701608,0.293848,0.579691,0.028801
2,2.003288,3.111346,2.906727,2.202060,0.293848,0.350663,0.020526
3,2.269460,2.599428,3.167186,2.437141,0.293848,0.579691,0.028801
4,1.703844,3.352249,3.167186,2.701608,0.293848,0.121636,0.000942


In [ ]:
# selecting non numeric features
nonnumeric_features = df_X.select_dtypes(include=object).columns.tolist()
nonnumeric_features

['Species', 'Country.of.Origin', 'Processing.Method']

In [ ]:
X_categorical= pd.get_dummies(df_X[nonnumeric_features], columns = nonnumeric_features, drop_first=True)
X_categorical = X_categorical.reset_index(drop=True)
X_categorical.head()


,Species_Robusta,Country.of.Origin_Burundi,Country.of.Origin_China,Country.of.Origin_Colombia,Country.of.Origin_Costa Rica,Country.of.Origin_Cote d?Ivoire,Country.of.Origin_Ecuador,Country.of.Origin_El Salvador,Country.of.Origin_Ethiopia,Country.of.Origin_Guatemala,...,Country.of.Origin_Thailand,Country.of.Origin_Uganda,Country.of.Origin_United States,Country.of.Origin_United States (Puerto Rico),Country.of.Origin_Vietnam,Country.of.Origin_Zambia,Processing.Method_Other,Processing.Method_Pulped natural / honey,Processing.Method_Semi-washed / Semi-pulped,Processing.Method_Washed / Wet
0,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
1,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
2,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# concatenating columns
X = pd.concat([X_n, X_categorical], join="inner" ,axis=1)
X.head()


,Aroma,Aftertaste,Acidity,Balance,Uniformity,Moisture,altitude_mean_meters,Species_Robusta,Country.of.Origin_Burundi,Country.of.Origin_China,...,Country.of.Origin_Thailand,Country.of.Origin_Uganda,Country.of.Origin_United States,Country.of.Origin_United States (Puerto Rico),Country.of.Origin_Vietnam,Country.of.Origin_Zambia,Processing.Method_Other,Processing.Method_Pulped natural / honey,Processing.Method_Semi-washed / Semi-pulped,Processing.Method_Washed / Wet
0,3.666864,3.864168,3.981118,2.701608,0.293848,0.579691,0.028801,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,3.933036,3.352249,3.427644,2.701608,0.293848,0.579691,0.028801,0,0,0,...,0,0,0,0,0,0,0,0,0,1
2,2.003288,3.111346,2.906727,2.202060,0.293848,0.350663,0.020526,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2.269460,2.599428,3.167186,2.437141,0.293848,0.579691,0.028801,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,1.703844,3.352249,3.167186,2.701608,0.293848,0.121636,0.000942,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# categorising been grade using percentile

been_grade = [1, 2, 3]
pctile = np.percentile(Y, [75, 90])

Y = np.where(Y < pctile[0], 1, Y)
Y = np.where((Y >= pctile[0]) & (Y <= pctile[1]), 2, Y)
Y = np.where(Y > pctile[1], 3, Y)

Y = pd.DataFrame(Y, columns=['been_grade'])
Y

,been_grade
0,3.0
1,3.0
2,3.0
3,3.0
4,3.0
...,...
1008,1.0
1009,1.0
1010,1.0
1011,1.0


Train and Test:

In [ ]:
state = 40
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.33, random_state = state)

Models:

In [ ]:
# function for all model
def train_test_evaluate(model_name, model, x_train, y_train, x_test, y_test):

    model.fit(x_train, y_train)
    y_pred= model.predict(x_test)

    if model_name == "Linear Regression" or "Random Forest":
      accuracy = r2_score(y_test, y_pred)
    else:
      accuracy = accuracy_score(y_test, y_pred)


    data=[[model_name, accuracy]]


    result_df = pd.DataFrame( data, columns=["Model", "Accuracy"])
    return result_df

In [ ]:
# models
# Decision tree

dt_model= DecisionTreeRegressor()

result_df = train_test_evaluate("Decision Tree", dt_model, x_train, y_train, x_test, y_test)
result_df

,Model,Accuracy
0,Decision Tree,0.648109


In [ ]:
# logistic regression
clf = LogisticRegression()
result_df2 = train_test_evaluate("Logistic Regression", clf, x_train, y_train, x_test, y_test)
result_df = result_df.append(result_df2)



/usr/local/lib/python3.10/dist-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
<ipython-input-61-1fb8865c59ba>:4: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_df2)


In [ ]:
# Linear Reg
ln = LinearRegression()

result_df2 = train_test_evaluate("Linear Regression", ln, x_train, y_train, x_test, y_test)
result_df = result_df.append(result_df2)


<ipython-input-62-26b023afc48a>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_df2)


In [ ]:
# Random Forest
rf = RandomForestRegressor()

result_df2 = train_test_evaluate("Random Forest", rf, x_train, y_train, x_test, y_test)
result_df = result_df.append(result_df2)

<ipython-input-59-b30aa64b76ad>:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  model.fit(x_train, y_train)
<ipython-input-63-9612acb588a3>:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_df = result_df.append(result_df2)


The best model:

In [ ]:
result_df

,Model,Accuracy
0,Decision Tree,0.648109
0,Logistic Regression,0.695028
0,Linear Regression,0.429938
0,Random Forest,0.746901
